In [ ]:
# !pip list

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from huggingface_hub import login, HfApi
import warnings
import logging
import os

# Suppress warnings
warnings.filterwarnings('ignore')
logging.getLogger("transformers").setLevel(logging.ERROR)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'

print("✓ All imports successful")

In [ ]:
# ============================================================================
# HUGGING FACE CONFIGURATION
# ============================================================================

class HF_CONFIG:
    """Hugging Face Hub configuration"""
    
    # Your Hugging Face username and model repository name
    HUB_USERNAME = "AKG2"  # ← Change this to your HF username
    HUB_MODEL_NAME = "clinical-longformer-readmission-stl"  # ← Change model name if desired
    
    # This will be constructed as: username/model-name
    @property
    def HUB_MODEL_ID(self):
        return f"{self.HUB_USERNAME}/{self.HUB_MODEL_NAME}"
    
    # Kaggle secret name for HF token (don't change unless you named it differently)
    KAGGLE_SECRET_NAME = "HF_TOKEN"
    
    # Whether to push to hub during training
    PUSH_TO_HUB = True
    
    # Whether to make the repository private
    PRIVATE_REPO = False


# ============================================================================
# IMPROVED TRAINING CONFIGURATION
# ============================================================================

class CONFIG:
    """Training configuration with improvements"""
    
    # Data paths
    DATA_PATH = "/kaggle/input/mimic-iv"
    MODEL_NAME = "yikuan8/Clinical-Longformer"  # Keep same base model
    
    # Hugging Face configuration
    HF = HF_CONFIG()
    
    # Dataset settings
    SAMPLE_SIZE = 10000  # Use full dataset for better training
    TEST_SIZE = 0.2
    RANDOM_SEED = 42
    
    # CRITICAL: Balance the dataset
    BALANCE_CLASSES = True  # NEW: Undersample majority class
    MAX_SAMPLES_PER_CLASS = 75000  # NEW: Limit to balance with minority
    
    # Model settings
    MAX_LENGTH = 2048  # CHANGED: Reduced from 4096 for better memory/speed
    
    # Training hyperparameters - MAJOR CHANGES
    EPOCHS = 10  # CHANGED: More epochs with early stopping
    TRAIN_BATCH_SIZE = 4  # CHANGED: Increased from 2
    VALID_BATCH_SIZE = 8  # CHANGED: Increased from 4
    GRADIENT_ACCUMULATION = 8  # CHANGED: Reduced from 16
    LEARNING_RATE = 5e-5  # CHANGED: Increased from 2e-5 for faster convergence
    WEIGHT_DECAY = 0.01
    WARMUP_RATIO = 0.05  # CHANGED: Reduced from 0.1 for faster warmup
    MAX_GRAD_NORM = 1.0  # NEW: Clip gradients to prevent instability
    
    # Optimizer settings
    ADAM_EPSILON = 1e-8  # NEW: For numerical stability
    ADAM_BETA1 = 0.9  # NEW
    ADAM_BETA2 = 0.999  # NEW
    
    # Early stopping
    EARLY_STOPPING_PATIENCE = 3  # NEW: Stop if no improvement
    
    # GPU optimization
    USE_FP16 = True
    
    # Output directory
    OUTPUT_DIR = "./stl_readmission_model_improved"

# ============================================================================
# HUGGING FACE HUB LOGIN
# ============================================================================

def login_to_huggingface(config):
    """Log in to Hugging Face Hub using Kaggle secret"""
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        hf_token = user_secrets.get_secret(config.HF.KAGGLE_SECRET_NAME)
        
        if hf_token:
            login(token=hf_token, add_to_git_credential=True)
            print(f"✓ Successfully logged in to Hugging Face Hub")
            print(f"  Model will be pushed to: {config.HF.HUB_MODEL_ID}")
            return True
        else:
            print("⚠ HF_TOKEN not found in Kaggle secrets")
            return False
    except Exception as e:
        print(f"⚠ Could not log in to Hugging Face Hub: {e}")
        print("  Training will continue without Hub integration")
        return False


# ============================================================================
# IMPROVED DATA LOADING WITH CLASS BALANCING
# ============================================================================

def load_and_prepare_data(config):
    """Load and merge all data sources with class balancing"""
    print("=" * 80)
    print("LOADING DATA")
    print("=" * 80)
    
    # Load main admissions file
    admissions = pd.read_csv(f"{config.DATA_PATH}/admissions_with_readmission_labels.csv")
    print(f"✓ Loaded admissions: {admissions.shape}")
    
    # Load discharge notes
    discharge = pd.read_csv(f"{config.DATA_PATH}/discharge_notes-001.csv")
    print(f"✓ Loaded discharge notes: {discharge.shape}")
    
    # Load radiology notes
    radiology = pd.read_csv(f"{config.DATA_PATH}/radiology_notes.csv")
    print(f"✓ Loaded radiology notes: {radiology.shape}")
    
    # Combine notes
    print("\nCombining notes...")
    discharge_grouped = discharge.groupby('hadm_id')['text'].apply(
        lambda x: ' '.join(x.astype(str))
    ).reset_index()
    discharge_grouped.columns = ['hadm_id', 'discharge_text']
    
    radiology_grouped = radiology.groupby('hadm_id')['text'].apply(
        lambda x: ' '.join(x.astype(str))
    ).reset_index()
    radiology_grouped.columns = ['hadm_id', 'radiology_text']
    
    # Merge notes
    notes_combined = discharge_grouped.merge(
        radiology_grouped, on='hadm_id', how='outer'
    )
    
    # Combine all text
    notes_combined['combined_text'] = (
        notes_combined['discharge_text'].fillna('') + ' ' + 
        notes_combined['radiology_text'].fillna('')
    )
    notes_combined['combined_text'] = notes_combined['combined_text'].str.strip()
    
    # Merge with admissions
    df = admissions.merge(notes_combined[['hadm_id', 'combined_text']], 
                          on='hadm_id', how='left')
    
    # Use combined_text if available
    df['final_text'] = df['combined_text'].fillna(df.get('text', ''))
    df['final_text'] = df['final_text'].fillna('')
    
    # Remove rows with empty text
    df = df[df['final_text'].str.len() > 0]
    print(f"✓ Final merged data: {df.shape}")
    
    # Prepare readmission labels
    print("\nPreparing labels...")
    df['readmitted_30day'] = df['readmitted_30day'].astype(int)
    
    print(f"  - Original distribution: {df['readmitted_30day'].value_counts().to_dict()}")
    
    # ========================================================================
    # CRITICAL: BALANCE THE DATASET
    # ========================================================================
    if config.BALANCE_CLASSES:
        print("\n⚖️  Balancing dataset...")
        
        # Separate positive and negative samples
        pos_samples = df[df['readmitted_30day'] == 1]
        neg_samples = df[df['readmitted_30day'] == 0]
        
        print(f"  - Positive samples: {len(pos_samples)}")
        print(f"  - Negative samples: {len(neg_samples)}")
        
        # Determine target size for each class
        target_size = min(
            len(pos_samples),
            len(neg_samples),
            config.MAX_SAMPLES_PER_CLASS
        )
        
        print(f"  - Target size per class: {target_size}")
        
        # Sample both classes to target size
        pos_samples = pos_samples.sample(n=target_size, random_state=config.RANDOM_SEED)
        neg_samples = neg_samples.sample(n=target_size, random_state=config.RANDOM_SEED)
        
        # Combine and shuffle
        df = pd.concat([pos_samples, neg_samples], ignore_index=True)
        df = df.sample(frac=1, random_state=config.RANDOM_SEED).reset_index(drop=True)
        
        print(f"  - Balanced dataset size: {len(df)}")
        print(f"  - New distribution: {df['readmitted_30day'].value_counts().to_dict()}")
    
    # Sample data if needed (for testing)
    if config.SAMPLE_SIZE is not None:
        df = df.sample(n=min(config.SAMPLE_SIZE, len(df)), 
                       random_state=config.RANDOM_SEED)
        print(f"✓ Sampled {len(df)} rows for testing")
    
    # Calculate class weights (should be 1.0 if balanced)
    pos_count = df['readmitted_30day'].sum()
    neg_count = len(df) - pos_count
    pos_weight = neg_count / pos_count if pos_count > 0 else 1.0
    
    print(f"\n  - Final positive weight: {pos_weight:.2f}")
    print(f"  - Final dataset size: {len(df)}")
    
    return df, pos_weight

# ============================================================================
# DATASET CLASS
# ============================================================================

class ReadmissionDataset(Dataset):
    """Dataset for single-task readmission prediction"""
    
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }


# ============================================================================
# IMPROVED SINGLE-TASK MODEL
# ============================================================================

class ReadmissionModel(nn.Module):
    """Improved single-task model for 30-day readmission prediction"""
    
    def __init__(self, model_name):
        super().__init__()
        
        # Backbone: Clinical-Longformer
        self.backbone = AutoModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size
        
        # IMPROVED: Better classification head with layer normalization
        self.dropout1 = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(hidden_size)  # NEW: Stabilizes training
        
        # NEW: Add intermediate layer for better representation
        self.intermediate = nn.Linear(hidden_size, hidden_size // 2)
        self.activation = nn.GELU()  # NEW: Better than ReLU for transformers
        self.dropout2 = nn.Dropout(0.2)
        
        # Final classifier
        self.classifier = nn.Linear(hidden_size // 2, 1)
        
        # Initialize weights properly
        self._init_weights()
    
    def _init_weights(self):
        """Initialize weights for better convergence"""
        nn.init.xavier_uniform_(self.intermediate.weight)
        nn.init.zeros_(self.intermediate.bias)
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)
    
    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        # Get [CLS] token representation from backbone
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Use [CLS] token (first token)
        cls_output = outputs.last_hidden_state[:, 0, :]
        
        # Apply improvements
        cls_output = self.dropout1(cls_output)
        cls_output = self.layer_norm(cls_output)  # Stabilize
        
        # Intermediate layer
        hidden = self.intermediate(cls_output)
        hidden = self.activation(hidden)
        hidden = self.dropout2(hidden)
        
        # Final prediction
        logits = self.classifier(hidden)
        
        return {'logits': logits}

# ============================================================================
# IMPROVED CUSTOM TRAINER
# ============================================================================

class ImprovedWeightedLossTrainer(Trainer):
    """Custom trainer with improvements for stability and convergence"""
    
    def __init__(self, pos_weight, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = torch.tensor([pos_weight])
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        
        # Track training metrics
        self.train_losses = []
        self.eval_metrics = []
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Move pos_weight to correct device
        model_device = next(model.parameters()).device
        if self.pos_weight.device != model_device:
            self.pos_weight = self.pos_weight.to(model_device)
            self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        
        # Extract labels
        labels = inputs.pop('labels')
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs['logits'].squeeze()
        
        # Calculate loss
        loss = self.bce_loss(logits, labels)
        
        # IMPROVEMENT: Add L2 regularization on logits to prevent extreme values
        l2_reg = 0.001 * torch.mean(logits ** 2)
        loss = loss + l2_reg
        
        # Track losses
        self.train_losses.append(loss.item())
        
        return (loss, outputs) if return_outputs else loss
    
    def log(self, logs, start_time=None):
        """Enhanced logging"""
        super().log(logs, start_time=start_time)
        
        # Print cleaner progress updates
        if 'loss' in logs:
            epoch = logs.get('epoch', 0)
            loss = logs.get('loss', 0)
            lr = logs.get('learning_rate', 0)
            grad_norm = logs.get('grad_norm', 0)
            
            print(f"Epoch {epoch:.2f} | Loss: {loss:.4f} | LR: {lr:.2e} | Grad: {grad_norm:.2f}")
        
        if 'eval_roc_auc' in logs:
            print(f"\n📊 Evaluation Results:")
            print(f"   ROC-AUC:   {logs['eval_roc_auc']:.4f}")
            print(f"   Accuracy:  {logs['eval_accuracy']:.4f}")
            print(f"   Precision: {logs['eval_precision']:.4f}")
            print(f"   Recall:    {logs['eval_recall']:.4f}")
            print(f"   F1 Score:  {logs['eval_f1']:.4f}")
            print(f"   Loss:      {logs['eval_loss']:.4f}\n")
            
            self.eval_metrics.append(logs)


# ============================================================================
# EARLY STOPPING CALLBACK
# ============================================================================

from transformers import TrainerCallback, TrainerState, TrainerControl

class EarlyStoppingCallback(TrainerCallback):
    """Early stopping callback to prevent overfitting"""
    
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_metric = None
        self.wait = 0
        self.stopped_epoch = 0
    
    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics, **kwargs):
        metric_value = metrics.get("eval_roc_auc")
        
        if metric_value is None:
            return control
        
        if self.best_metric is None:
            self.best_metric = metric_value
        elif metric_value > self.best_metric + self.min_delta:
            self.best_metric = metric_value
            self.wait = 0
            print(f"✓ New best ROC-AUC: {metric_value:.4f}")
        else:
            self.wait += 1
            print(f"⚠️  No improvement for {self.wait} epoch(s)")
            
            if self.wait >= self.patience:
                self.stopped_epoch = state.epoch
                control.should_training_stop = True
                print(f"\n🛑 Early stopping triggered after {self.wait} epochs without improvement")
        
        return control
        
# ============================================================================
# EVALUATION METRICS
# ============================================================================

def compute_metrics(eval_pred):
    """Compute metrics for readmission prediction"""
    predictions, labels = eval_pred
    
    # Extract logits (handle both dict and tensor formats)
    if isinstance(predictions, dict):
        logits = predictions['logits']
    elif isinstance(predictions, tuple):
        logits = predictions[0]
    else:
        logits = predictions
    
    # Convert to numpy if needed
    logits_np = logits.numpy() if isinstance(logits, torch.Tensor) else logits
    
    # Convert logits to probabilities (sigmoid)
    probs = 1 / (1 + np.exp(-logits_np.squeeze()))
    preds = (probs > 0.5).astype(int)
    
    # Handle edge case: if all predictions are same class
    try:
        roc_auc = roc_auc_score(labels, probs)
    except ValueError:
        roc_auc = 0.0
    
    # Calculate metrics
    metrics = {
        'roc_auc': roc_auc,
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, zero_division=0),
        'recall': recall_score(labels, preds, zero_division=0),
        'f1': f1_score(labels, preds, zero_division=0)
    }
    
    return metrics


# ============================================================================
# CHECK FOR EXISTING MODEL ON HUB
# ============================================================================

def check_hub_model_exists(config):
    """Check if model already exists on Hugging Face Hub"""
    try:
        api = HfApi()
        repos = api.list_models(author=config.HF.HUB_USERNAME)
        model_exists = any(repo.modelId == config.HF.HUB_MODEL_ID for repo in repos)
        
        if model_exists:
            print(f"✓ Found existing model on Hub: {config.HF.HUB_MODEL_ID}")
            return True
        else:
            print(f"  No existing model found on Hub, will create new repository")
            return False
    except Exception as e:
        print(f"  Could not check Hub for existing model: {e}")
        return False

In [ ]:
print("\n" + "=" * 80)
print("SINGLE-TASK LEARNING FOR HOSPITAL READMISSION PREDICTION")
print("=" * 80 + "\n")

# Set random seeds
torch.manual_seed(CONFIG.RANDOM_SEED)
np.random.seed(CONFIG.RANDOM_SEED)

# Login to Hugging Face Hub
hf_logged_in = login_to_huggingface(CONFIG)

# Check if model exists on Hub
model_exists_on_hub = False
if hf_logged_in and CONFIG.HF.PUSH_TO_HUB:
    model_exists_on_hub = check_hub_model_exists(CONFIG)

In [ ]:
# Load data
df, pos_weight = load_and_prepare_data(CONFIG)

# Split data
print("\n" + "=" * 80)
print("SPLITTING DATA")
print("=" * 80)
train_df, val_df = train_test_split(
    df, 
    test_size=CONFIG.TEST_SIZE, 
    random_state=CONFIG.RANDOM_SEED,
    stratify=df['readmitted_30day']
)
print(f"✓ Train size: {len(train_df)}")
print(f"✓ Validation size: {len(val_df)}")

In [ ]:
# Load tokenizer
print("\n" + "=" * 80)
print("LOADING TOKENIZER")
print("=" * 80)

# Try to load from Hub first if model exists there
tokenizer_source = CONFIG.MODEL_NAME
if model_exists_on_hub:
    try:
        tokenizer = AutoTokenizer.from_pretrained(CONFIG.HF.HUB_MODEL_ID)
        tokenizer_source = CONFIG.HF.HUB_MODEL_ID
        print(f"✓ Loaded tokenizer from Hub: {tokenizer_source}")
    except:
        tokenizer = AutoTokenizer.from_pretrained(CONFIG.MODEL_NAME)
        print(f"✓ Loaded tokenizer from base model: {tokenizer_source}")
else:
    tokenizer = AutoTokenizer.from_pretrained(CONFIG.MODEL_NAME)
    print(f"✓ Loaded tokenizer: {tokenizer_source}")

# Create datasets
print("\n" + "=" * 80)
print("CREATING DATASETS")
print("=" * 80)
train_dataset = ReadmissionDataset(
    texts=train_df['final_text'].values,
    labels=train_df['readmitted_30day'].values,
    tokenizer=tokenizer,
    max_length=CONFIG.MAX_LENGTH
)

val_dataset = ReadmissionDataset(
    texts=val_df['final_text'].values,
    labels=val_df['readmitted_30day'].values,
    tokenizer=tokenizer,
    max_length=CONFIG.MAX_LENGTH
)
print(f"✓ Train dataset: {len(train_dataset)} samples")
print(f"✓ Validation dataset: {len(val_dataset)} samples")

In [ ]:
# ============================================================================
# IMPROVED TRAINING SETUP AND EXECUTION
# ============================================================================

# Initialize model
print("\n" + "=" * 80)
print("INITIALIZING IMPROVED MODEL")
print("=" * 80)

model = ReadmissionModel(CONFIG.MODEL_NAME)
print(f"✓ Initialized model from: {CONFIG.MODEL_NAME}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  - Total parameters: {total_params:,}")
print(f"  - Trainable parameters: {trainable_params:,}")

# IMPROVED: Training arguments with better hyperparameters
training_args = TrainingArguments(
    output_dir=CONFIG.OUTPUT_DIR,
    num_train_epochs=CONFIG.EPOCHS,
    per_device_train_batch_size=CONFIG.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=CONFIG.VALID_BATCH_SIZE,
    gradient_accumulation_steps=CONFIG.GRADIENT_ACCUMULATION,
    
    # Learning rate and optimization
    learning_rate=CONFIG.LEARNING_RATE,
    weight_decay=CONFIG.WEIGHT_DECAY,
    warmup_ratio=CONFIG.WARMUP_RATIO,
    max_grad_norm=CONFIG.MAX_GRAD_NORM,  # NEW: Gradient clipping
    
    # Optimizer settings
    adam_epsilon=CONFIG.ADAM_EPSILON,  # NEW
    adam_beta1=CONFIG.ADAM_BETA1,  # NEW
    adam_beta2=CONFIG.ADAM_BETA2,  # NEW
    
    # Mixed precision
    fp16=CONFIG.USE_FP16,
    
    # Logging and evaluation
    logging_steps=50,  # CHANGED: Less frequent logging
    logging_first_step=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    
    # Model selection
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    save_total_limit=3,  # Keep top 3 checkpoints
    
    # Other settings
    report_to="none",
    seed=CONFIG.RANDOM_SEED,
    disable_tqdm=True,
    
    # Hugging Face Hub
    push_to_hub=CONFIG.HF.PUSH_TO_HUB and hf_logged_in,
    hub_model_id=CONFIG.HF.HUB_MODEL_ID if hf_logged_in else None,
    hub_strategy="end",  # CHANGED: Only push at end
    hub_private_repo=CONFIG.HF.PRIVATE_REPO,
)

# Initialize callbacks
callbacks = [
    EarlyStoppingCallback(
        patience=CONFIG.EARLY_STOPPING_PATIENCE,
        min_delta=0.001
    )
]

# Initialize trainer
print("\n" + "=" * 80)
print("INITIALIZING IMPROVED TRAINER")
print("=" * 80)

trainer = ImprovedWeightedLossTrainer(
    pos_weight=pos_weight,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=callbacks,
)

print("✓ Trainer initialized with improvements:")
print(f"  - Effective batch size: {CONFIG.TRAIN_BATCH_SIZE * CONFIG.GRADIENT_ACCUMULATION}")
print(f"  - Learning rate: {CONFIG.LEARNING_RATE}")
print(f"  - Gradient clipping: {CONFIG.MAX_GRAD_NORM}")
print(f"  - Early stopping patience: {CONFIG.EARLY_STOPPING_PATIENCE}")
print(f"  - Max length: {CONFIG.MAX_LENGTH}")

# Check for existing checkpoints
checkpoint_path = None
if os.path.exists(CONFIG.OUTPUT_DIR):
    checkpoints = [
        os.path.join(CONFIG.OUTPUT_DIR, d) 
        for d in os.listdir(CONFIG.OUTPUT_DIR) 
        if d.startswith("checkpoint-")
    ]
    if checkpoints:
        checkpoint_path = max(checkpoints, key=os.path.getmtime)
        print(f"\n✓ Found existing checkpoint: {checkpoint_path}")
        print("  Training will resume from this checkpoint")

# Train
print("\n" + "=" * 80)
print("TRAINING MODEL WITH IMPROVEMENTS")
print("=" * 80)
print("Expected improvements:")
print("  ✓ Balanced dataset (equal class distribution)")
print("  ✓ Higher learning rate for faster convergence")
print("  ✓ Gradient clipping to prevent instability")
print("  ✓ Better model architecture with intermediate layer")
print("  ✓ Early stopping to prevent overfitting")
print("=" * 80 + "\n")

if checkpoint_path:
    trainer.train(resume_from_checkpoint=checkpoint_path)
else:
    trainer.train()

# Print training summary
print("\n" + "=" * 80)
print("TRAINING SUMMARY")
print("=" * 80)

if hasattr(trainer, 'train_losses') and len(trainer.train_losses) > 0:
    avg_train_loss = sum(trainer.train_losses) / len(trainer.train_losses)
    print(f"Average training loss: {avg_train_loss:.4f}")
    print(f"Final training loss: {trainer.train_losses[-1]:.4f}")

if hasattr(trainer, 'eval_metrics') and len(trainer.eval_metrics) > 0:
    best_eval = max(trainer.eval_metrics, key=lambda x: x.get('eval_roc_auc', 0))
    print(f"\nBest evaluation metrics:")
    print(f"  ROC-AUC:   {best_eval['eval_roc_auc']:.4f}")
    print(f"  Accuracy:  {best_eval['eval_accuracy']:.4f}")
    print(f"  F1 Score:  {best_eval['eval_f1']:.4f}")


In [ ]:
# Final evaluation
print("\n" + "=" * 80)
print("FINAL EVALUATION ON BEST MODEL")
print("=" * 80)

eval_results = trainer.evaluate()

print("\n📊 FINAL RESULTS:")
print(f"  ROC-AUC:   {eval_results['eval_roc_auc']:.4f}")
print(f"  Accuracy:  {eval_results['eval_accuracy']:.4f}")
print(f"  Precision: {eval_results['eval_precision']:.4f}")
print(f"  Recall:    {eval_results['eval_recall']:.4f}")
print(f"  F1 Score:  {eval_results['eval_f1']:.4f}")
print(f"  Loss:      {eval_results['eval_loss']:.4f}")

print("\n" + "=" * 80)
print("TRAINING COMPLETE!")
print("=" * 80)